# Step II: Energy dynamics simulation
After having fixed the optimum insulation thickness value, with Besos the .idf file is edited by setting the found value, then with Eppy, different simulations are performed to obtained different annual temperature and HVAC consumption profiles.

In [1]:
import matplotlib.pyplot as plt
from seaborn import heatmap
from seaborn import pairplot
from besos import eppy_funcs as ef
from besos import sampling
from besos.evaluator import EvaluatorEP
from besos.parameters import RangeParameter, FieldSelector, FilterSelector, Parameter, expand_plist, wwr, CategoryParameter, GenericSelector
from besos.problem import EPProblem
import pandas as pd
from eppy import modeleditor
from eppy.modeleditor import IDF
import os

MAT = .25
WWR = .9
GL = 2.85

iddfile='/usr/local/EnergyPlus-9-0-1/Energy+.idd'
weather='../model/epw/RUS_Moscow.276120_IWEC.epw'

# Load the building from the IDF file
building = ef.get_building(f'../model/idf/9.0/model_2_1.idf')

# Define the problem
insulation = FieldSelector(
    class_name='Material', 
    object_name='Rock wool - unbonded_01', 
    field_name='Thickness'
)
glazing1 = FieldSelector(
    class_name='WindowMaterial:Glazing',
    object_name='2',
    field_name='Thickness'
)
glazing2 = FieldSelector(
    class_name='WindowMaterial:Glazing',
    object_name='40',
    field_name='Thickness'
)
#==========================================
#PARAMETERS
#==========================================
parameters = [
    Parameter(
        selector=insulation,
        #value_descriptor = RangeParameter(0.03,0.36),
        value_descriptor = RangeParameter(0.0,0.4),
        name='Insulation Thickness'
    ),
    wwr(CategoryParameter(options=[.15,.5,.9])),
    Parameter(
        selector=glazing1,
        value_descriptor = RangeParameter(.7, 5),
        name='Glazing1'
    ),
    Parameter(
        selector=glazing2,
        value_descriptor = RangeParameter(.7, 5),
        name='Glazing2'
    )
]
ef.wwr_all(building, WWR)
#================================
#OBJECTIVES AND PROBLEM
#================================
# Define the problem's objectives
objectives = ['Electricity:Facility']

problem=EPProblem(parameters, objectives)

# Set the evaluator
evaluator = EvaluatorEP(
    problem, 
    building,
    weather,
    out_dir='outputdir', 
    err_dir='errdir',
)

samples = pd.DataFrame({
    'Insulation Thickness':[MAT],
    'Window to Wall Ratio':[WWR],
    'Glazing1':[GL],
    'Glazing2':[GL]
})

print('Performing evaluation...')
evaluator.df_apply(samples, keep_input=True)


/usr/local/lib/python3.7/dist-packages/besos/parameters.py:400: UserWarning: wwr is intended to be used with RangeParameter. Your value_descriptor is CategoryParameter(options=[0.15, 0.5, 0.9])
  warnings.warn(f'wwr is intended to be used with RangeParameter. Your value_descriptor is {value_descriptor}')


Performing evaluation...


,Insulation Thickness,Window to Wall Ratio,Glazing1,Glazing2,Electricity:Facility
0,0.25,0.9,2.85,2.85,3.056227e+10


In [2]:
print('Running e+ simulation...')

#IDF.setiddname(iddfile)
idf = IDF(f'outputdir/BESOS_Output/in.idf',weather)

idf.run(readvars=True)

os.system(f'cp eplusout.csv ../data/ensig/epraw.csv')

Running e+ simulation...

/usr/local/EnergyPlus-9-0-1/energyplus --weather /home/luca/Codes/ICT4BD/model/epw/RUS_Moscow.276120_IWEC.epw --output-directory /home/luca/Codes/ICT4BD/step2 --idd /usr/local/lib/python3.7/dist-packages/Data/example_idd.idd --readvars /home/luca/Codes/ICT4BD/step2/in.idf



0